## Config and Imports

In [56]:
import pandas as pd 
import numpy as np
import sys
import os
import seaborn as sns
import matplotlib.pyplot as plt

## Tratamento e Análise da tabela "event tracks"

In [57]:
event_tracks = pd.read_csv(r'../01. Raw data\DadosBrutosEventTracks.csv', sep=';')

In [58]:
event_tracks['date'] = pd.to_datetime(event_tracks['StorageDate'], utc=True, errors='coerce')
event_tracks['date'] = event_tracks['date'].dt.tz_convert('America/Sao_Paulo')
event_tracks['date'] = event_tracks['date'].dt.tz_localize(None).dt.floor('s')
event_tracks

,EventTrackSequentialID,OwnerIdentity,StorageDate,Category,Action,date
0,100,carnabelo@msging.net,2018-02-02 13:51:59.72 -02:00,Flow,Boas vindas,2018-02-02 13:51:59
1,101,carnabelo@msging.net,2018-02-02 13:52:10.89 -02:00,Flow,Objetivo - Beber,2018-02-02 13:52:10
2,102,carnabelo@msging.net,2018-02-02 13:52:10.98 -02:00,Objetivo,Beber,2018-02-02 13:52:10
3,103,carnabelo@msging.net,2018-02-02 13:52:13.15 -02:00,Flow,Intro,2018-02-02 13:52:13
4,104,carnabelo@msging.net,2018-02-02 13:52:17.34 -02:00,Flow,Menu,2018-02-02 13:52:17
...,...,...,...,...,...,...
52895,62980,carnabelo@msging.net,2018-02-27 09:16:34.80 -03:00,Flow,API - Carrossel blocos - Agenda,2018-02-27 09:16:34
52896,62981,carnabelo@msging.net,2018-02-27 09:16:34.87 -03:00,Pesquisa - Bairro,Santa Rosa,2018-02-27 09:16:34
52897,62982,carnabelo@msging.net,2018-02-27 10:17:09.71 -03:00,Flow,Respostas para carrossel,2018-02-27 10:17:09
52898,62983,carnabelo@msging.net,2018-02-27 10:17:09.87 -03:00,Flow,Compartilhar - Intro,2018-02-27 10:17:09


In [59]:
event_tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52900 entries, 0 to 52899
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   EventTrackSequentialID  52900 non-null  int64         
 1   OwnerIdentity           52900 non-null  object        
 2   StorageDate             52900 non-null  object        
 3   Category                52900 non-null  object        
 4   Action                  52900 non-null  object        
 5   date                    52900 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 2.4+ MB


In [60]:
#Selecionando colunas pertinentes
event_tracks_short = event_tracks[['EventTrackSequentialID','Category', 'Action', 'date']].copy()
event_tracks_short

,EventTrackSequentialID,Category,Action,date
0,100,Flow,Boas vindas,2018-02-02 13:51:59
1,101,Flow,Objetivo - Beber,2018-02-02 13:52:10
2,102,Objetivo,Beber,2018-02-02 13:52:10
3,103,Flow,Intro,2018-02-02 13:52:13
4,104,Flow,Menu,2018-02-02 13:52:17
...,...,...,...,...
52895,62980,Flow,API - Carrossel blocos - Agenda,2018-02-27 09:16:34
52896,62981,Pesquisa - Bairro,Santa Rosa,2018-02-27 09:16:34
52897,62982,Flow,Respostas para carrossel,2018-02-27 10:17:09
52898,62983,Flow,Compartilhar - Intro,2018-02-27 10:17:09


In [61]:
event_category_analysis = event_tracks.copy()
event_category_analysis = event_category_analysis[['Category']].value_counts().reset_index()
event_category_analysis.columns = ['Category', 'Count']
event_category_analysis = event_category_analysis.sort_values(by=['Count'], ascending=False)
event_category_analysis['importance'] = ((event_category_analysis['Count'] / event_category_analysis['Count'].sum())*100).round(2)
event_category_analysis['acum_sum'] = event_category_analysis['importance'].cumsum().round(2)
event_category_analysis

,Category,Count,importance,acum_sum
0,Flow,44727,84.55,84.55
1,Pesquisa - Bairro,2498,4.72,89.27
2,Pesquisa - Dia,1776,3.36,92.63
3,Eu vou,1011,1.91,94.54
4,Como chegar,843,1.59,96.13
5,Objetivo,785,1.48,97.61
6,Ads,384,0.73,98.34
7,broad-Pesquisa_Satisfacao,286,0.54,98.88
8,broad-Fim_Carnaval,182,0.34,99.22
9,Envio de foto,128,0.24,99.46


In [62]:
#Exploring data
# event_tracks[event_tracks.Category == 'Eu vou']
# event_tracks[event_tracks.Action == 'IA - Sentimento']
# event_tracks[event_tracks.Category == 'Pesquisa - Dia']
# event_tracks[event_tracks.EventTrackSequentialID <= 1896]
event_tracks['EventTrackSequentialID'].nunique()

52900

In [63]:
# Action Flow Analysis
event_tracks_action_flow_analysis = event_tracks[event_tracks['Category'] == 'Flow'].copy()
event_tracks_action_flow_analysis['fallback'] = np.where(event_tracks_action_flow_analysis['Action'].str.lower().str.contains('fallback'), True, False)
event_tracks_action_flow_analysis['fallback'] = event_tracks_action_flow_analysis['fallback'].astype(bool)
event_tracks_action_flow_analysis = event_tracks_action_flow_analysis.Action.value_counts().reset_index()
event_tracks_action_flow_analysis.columns = ['Action', 'Count']
event_tracks_action_flow_analysis['importance'] = (((event_tracks_action_flow_analysis['Count'] / event_tracks_action_flow_analysis['Count'].sum()))*100).round(2)
event_tracks_action_flow_analysis ['acum_sum'] = event_tracks_action_flow_analysis['importance'].cumsum().round(2)
event_tracks_action_flow_analysis


,Action,Count,importance,acum_sum
0,Respostas para carrossel,4369,9.77,9.77
1,API - Carrossel blocos - Agenda,2498,5.58,15.35
2,Fallback - Diversos,1967,4.40,19.75
3,Fallback - IA,1936,4.33,24.08
4,Selecionar data,1918,4.29,28.37
...,...,...,...,...
107,API - Carrossel blocos - Aconteceu,1,0.00,99.98
108,Emoji - Indefinido,1,0.00,99.98
109,Cancelar - Envios,1,0.00,99.98
110,API - Carrossel blocos - Tarde,1,0.00,99.98


In [64]:
event_tracks_short.to_csv(r'../02. Processed data\event_tracks_short.csv', sep=';', index=False)

## Deep Dive - Entendendo o objetivo do user ao interagir com o chatbot

In [65]:
# Extract action list and pre process
def pre_process(text):
    text = text.lower()
    remove_terms = ['api','fallback','ia -' ,'intro -','-', 'carrossel', ' ia']
    for term in remove_terms:
        text = text.replace(term, '')
    return text.strip()
raw_actions = event_tracks_action_flow_analysis['Action'].unique().tolist()
raw_actions = [item for item in raw_actions if item !='Fallback  - IA']  # Remove string that when processesed is empty - won't be clustered
processed_actions = event_tracks_action_flow_analysis['Action'].apply(pre_process).unique().tolist()
processed_actions= [item for item in processed_actions if item]

len(raw_actions)
len (processed_actions)
processed_actions

['respostas para',
 'blocos   agenda',
 'diversos',
 'selecionar data',
 'musica',
 'buscar info do bloco',
 'menu',
 'bairro',
 'nome de bloco',
 'consultar blocos hoje',
 'salvar  bloco',
 'verificar  horario bloco',
 'criar lista de distribuicao',
 'boas vindas',
 'consultar proximos blocos',
 'intro',
 'como chegar',
 'o que tem hoje',
 'blocos   hoje',
 'verificar  nome do bloco',
 'bloco  agendado',
 'agendar  push',
 'verificar  nome do  bairro',
 'permissao para envio de alerta',
 'erro padrao',
 'bairro  hoje  blocos',
 'enviar localizacao',
 'bot',
 'bairro  hoje',
 'objetivo  dancar',
 'medialink',
 'consultar  blocos   localizacao',
 'blocos   localizacao',
 'sentimento  positivo',
 'blocos   nome',
 'oquevaiter',
 'objetivo  tudo',
 'retornar imagem',
 'sentimento',
 'consultar  meus blocos',
 'bloco  ja comecou',
 'sentimento  negativo  pos',
 'indicar  enviar foto',
 'sentimento  amor',
 'sentimento  negativo',
 'permissao para publicar',
 'verificar  saudacao',
 'saudac

In [66]:
event_tracks_action_flow_analysis['processed_action'] = event_tracks_action_flow_analysis['Action'].apply(pre_process)
event_tracks_action_flow_analysis

,Action,Count,importance,acum_sum,processed_action
0,Respostas para carrossel,4369,9.77,9.77,respostas para
1,API - Carrossel blocos - Agenda,2498,5.58,15.35,blocos agenda
2,Fallback - Diversos,1967,4.40,19.75,diversos
3,Fallback - IA,1936,4.33,24.08,
4,Selecionar data,1918,4.29,28.37,selecionar data
...,...,...,...,...,...
107,API - Carrossel blocos - Aconteceu,1,0.00,99.98,blocos aconteceu
108,Emoji - Indefinido,1,0.00,99.98,emoji indefinido
109,Cancelar - Envios,1,0.00,99.98,cancelar envios
110,API - Carrossel blocos - Tarde,1,0.00,99.98,blocos tarde


In [67]:
def map_objective(action):
    if 'permissao' in action.lower():
        return 'servicos_bot'
    elif 'imagem' in action.lower():
        return 'servicos_bot'
    elif 'foto' in action.lower():
        return 'servicos_bot'
    elif 'playlist' in action.lower():
        return 'servicos_bot'
    elif 'meus blocos' in action.lower():
        return 'servicos_bot'
    elif 'dicas' in action.lower():
        return 'servicos_bot'
    elif 'compartilhar' in action.lower():
        return 'servicos_bot'
    elif 'agendado' in action.lower():
        return 'servicos_bot'
    elif 'salv' in action.lower():
        return 'servicos_bot'
    elif 'lista de distribuicao' in action.lower():
        return 'servicos_bot'
    
    elif 'data' in action.lower():
        return 'programacao_bloco'
    elif 'hoje' in action.lower():
        return 'programacao_bloco'
    elif 'amanha' in action.lower():
        return 'programacao_bloco'
    elif 'horario' in action.lower():
        return 'programacao_bloco'
    elif 'agenda' in action.lower():
        return 'programacao_bloco'
    elif 'OQueVaiTer' in action.lower():
        return 'programacao_bloco'
    elif 'quando' in action.lower():
        return 'programacao_bloco'
    elif 'dia' in action.lower():
        return 'programacao_bloco'
    elif 'tarde' in action.lower():
        return 'programacao_bloco'
    elif 'noite' in action.lower():
        return 'programacao_bloco'
    
    elif 'chegar' in action.lower():
        return 'localizacao_bloco'
    elif 'bairro' in action.lower():
        return 'localizacao_bloco'
    elif 'localizacao' in action.lower():
        return 'localizacao_bloco'
    
    elif 'broad' in action.lower():
        return 'lista_transmissao_bot'
    
    elif 'bloco' in action.lower():
        return 'informacao_bloco'
    else:
        return 'interacao_chatbot'

event_tracks_action_flow_analysis['objective'] = (event_tracks_action_flow_analysis['Action'].apply(map_objective))
event_tracks_action_flow_analysis

,Action,Count,importance,acum_sum,processed_action,objective
0,Respostas para carrossel,4369,9.77,9.77,respostas para,interacao_chatbot
1,API - Carrossel blocos - Agenda,2498,5.58,15.35,blocos agenda,programacao_bloco
2,Fallback - Diversos,1967,4.40,19.75,diversos,interacao_chatbot
3,Fallback - IA,1936,4.33,24.08,,interacao_chatbot
4,Selecionar data,1918,4.29,28.37,selecionar data,programacao_bloco
...,...,...,...,...,...,...
107,API - Carrossel blocos - Aconteceu,1,0.00,99.98,blocos aconteceu,informacao_bloco
108,Emoji - Indefinido,1,0.00,99.98,emoji indefinido,interacao_chatbot
109,Cancelar - Envios,1,0.00,99.98,cancelar envios,interacao_chatbot
110,API - Carrossel blocos - Tarde,1,0.00,99.98,blocos tarde,programacao_bloco


In [68]:
flow_actions_summary = event_tracks_action_flow_analysis.groupby('objective').agg({'Count': 'sum', 'importance': 'sum'}).reset_index().sort_values(by='importance', ascending=False).round(2)
flow_actions_summary

,objective,Count,importance
1,interacao_chatbot,17636,39.43
4,programacao_bloco,10836,24.22
0,informacao_bloco,5922,13.22
5,servicos_bot,5763,12.89
3,localizacao_bloco,4406,9.85
2,lista_transmissao_bot,164,0.37


In [69]:
### Exporting the results
event_tracks_action_flow_analysis.to_excel(r'../03. Exported results\event_tracks_action_flow_analysis.xlsx', index=False)
flow_actions_summary.to_excel(r'../03. Exported results\flow_actions_summary.xlsx', index=False)
event_category_analysis.to_excel(r'../03. Exported results\event_category_analysis.xlsx', index=False)

## Tratamento e Análise da tabela "messages"

### Tratamento básico

In [70]:
messages = pd.read_csv(r'../01. Raw data\DadosBrutosMessages.csv', sep=';')

In [71]:
messages['date'] = pd.to_datetime(messages['StorageDate'], utc = True, errors='coerce')
messages['date'] = messages['date'].dt.tz_convert('America/Sao_Paulo')
messages['date'] = messages['date'].dt.tz_localize(None).dt.floor('s')
messages.sort_values(by= ['MessageSequentialID'],ascending=True)

,MessageSequentialID,StorageDate,FromIdentity,ToIdentity,Message,MessageOriginator,date
0,73719,2018-02-02 13:51:58.78 -02:00,1454345038013757@messenger.gw.msging.net,carnabelo@msging.net,comecar,User,2018-02-02 13:51:58
1,73722,2018-02-02 13:52:00.92 -02:00,carnabelo@msging.net,1454345038013757@messenger.gw.msging.net,"E ai Alan Pierre, o carnaval de BH já começou 🎉 🎭",Bot,2018-02-02 13:52:00
2,73723,2018-02-02 13:52:03.90 -02:00,carnabelo@msging.net,1454345038013757@messenger.gw.msging.net,"Se você não sabe onde ir, posso te ajudar com ...",Bot,2018-02-02 13:52:03
3,73726,2018-02-02 13:52:10.64 -02:00,1454345038013757@messenger.gw.msging.net,carnabelo@msging.net,beber,User,2018-02-02 13:52:10
4,73729,2018-02-02 13:52:12.07 -02:00,carnabelo@msging.net,1454345038013757@messenger.gw.msging.net,🎵 Você pensa que cachaça é água?\nCachaça não ...,Bot,2018-02-02 13:52:12
...,...,...,...,...,...,...,...
33761,650957,2018-02-27 09:16:35.93 -03:00,carnabelo@msging.net,1897100100309538@messenger.gw.msging.net,Não tenho dúvidas que deu para curtir muito n...,Bot,2018-02-27 09:16:35
33762,650960,2018-02-27 09:16:36.93 -03:00,carnabelo@msging.net,1897100100309538@messenger.gw.msging.net,Olha os blocos que passaram por lá 👀,Bot,2018-02-27 09:16:36
33763,651746,2018-02-27 10:17:09.40 -03:00,1897100100309538@messenger.gw.msging.net,carnabelo@msging.net,Compartilhar c/geral 🎉,User,2018-02-27 10:17:09
33764,651749,2018-02-27 10:17:11.96 -03:00,carnabelo@msging.net,1897100100309538@messenger.gw.msging.net,Chama sua galera para o Cai na Folia.,Bot,2018-02-27 10:17:11


In [72]:
messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33766 entries, 0 to 33765
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   MessageSequentialID  33766 non-null  int64         
 1   StorageDate          33766 non-null  object        
 2   FromIdentity         33766 non-null  object        
 3   ToIdentity           33766 non-null  object        
 4   Message              33766 non-null  object        
 5   MessageOriginator    33766 non-null  object        
 6   date                 33766 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 1.8+ MB


In [73]:
#Tratamento da tabela para identificar o user em cada mensagem de interação com o bot, bem como a sequência contínua de mensagens de cada user com o bot
message_trat = messages[['MessageSequentialID','FromIdentity', 'ToIdentity', 'Message', 'MessageOriginator', 'date', 'StorageDate' ]].copy()
message_trat = message_trat.sort_values(by=['MessageSequentialID'], ascending=True)
message_trat['user'] = np.where(message_trat['FromIdentity'] == 'carnabelo@msging.net', message_trat['ToIdentity'], message_trat['FromIdentity'])
message_trat['message_sequence_number'] = message_trat.groupby('user')['MessageSequentialID'].rank(method='dense').astype(int)
message_enxuta = message_trat.drop(columns=['FromIdentity', 'ToIdentity', 'StorageDate'])
message_trat

,MessageSequentialID,FromIdentity,ToIdentity,Message,MessageOriginator,date,StorageDate,user,message_sequence_number
0,73719,1454345038013757@messenger.gw.msging.net,carnabelo@msging.net,comecar,User,2018-02-02 13:51:58,2018-02-02 13:51:58.78 -02:00,1454345038013757@messenger.gw.msging.net,1
1,73722,carnabelo@msging.net,1454345038013757@messenger.gw.msging.net,"E ai Alan Pierre, o carnaval de BH já começou 🎉 🎭",Bot,2018-02-02 13:52:00,2018-02-02 13:52:00.92 -02:00,1454345038013757@messenger.gw.msging.net,2
2,73723,carnabelo@msging.net,1454345038013757@messenger.gw.msging.net,"Se você não sabe onde ir, posso te ajudar com ...",Bot,2018-02-02 13:52:03,2018-02-02 13:52:03.90 -02:00,1454345038013757@messenger.gw.msging.net,3
3,73726,1454345038013757@messenger.gw.msging.net,carnabelo@msging.net,beber,User,2018-02-02 13:52:10,2018-02-02 13:52:10.64 -02:00,1454345038013757@messenger.gw.msging.net,4
4,73729,carnabelo@msging.net,1454345038013757@messenger.gw.msging.net,🎵 Você pensa que cachaça é água?\nCachaça não ...,Bot,2018-02-02 13:52:12,2018-02-02 13:52:12.07 -02:00,1454345038013757@messenger.gw.msging.net,5
...,...,...,...,...,...,...,...,...,...
33761,650957,carnabelo@msging.net,1897100100309538@messenger.gw.msging.net,Não tenho dúvidas que deu para curtir muito n...,Bot,2018-02-27 09:16:35,2018-02-27 09:16:35.93 -03:00,1897100100309538@messenger.gw.msging.net,13
33762,650960,carnabelo@msging.net,1897100100309538@messenger.gw.msging.net,Olha os blocos que passaram por lá 👀,Bot,2018-02-27 09:16:36,2018-02-27 09:16:36.93 -03:00,1897100100309538@messenger.gw.msging.net,14
33763,651746,1897100100309538@messenger.gw.msging.net,carnabelo@msging.net,Compartilhar c/geral 🎉,User,2018-02-27 10:17:09,2018-02-27 10:17:09.40 -03:00,1897100100309538@messenger.gw.msging.net,15
33764,651749,carnabelo@msging.net,1897100100309538@messenger.gw.msging.net,Chama sua galera para o Cai na Folia.,Bot,2018-02-27 10:17:11,2018-02-27 10:17:11.96 -03:00,1897100100309538@messenger.gw.msging.net,16


In [74]:
bot_messages = message_enxuta[message_enxuta.MessageOriginator == 'Bot']
bot_messages

,MessageSequentialID,Message,MessageOriginator,date,user,message_sequence_number
1,73722,"E ai Alan Pierre, o carnaval de BH já começou 🎉 🎭",Bot,2018-02-02 13:52:00,1454345038013757@messenger.gw.msging.net,2
2,73723,"Se você não sabe onde ir, posso te ajudar com ...",Bot,2018-02-02 13:52:03,1454345038013757@messenger.gw.msging.net,3
4,73729,🎵 Você pensa que cachaça é água?\nCachaça não ...,Bot,2018-02-02 13:52:12,1454345038013757@messenger.gw.msging.net,5
5,73730,"Beber pode? Claro que pode!! Se for catuaçaÍ, ...",Bot,2018-02-02 13:52:13,1454345038013757@messenger.gw.msging.net,6
6,73732,Tenho essas opções para te ajudar a encontrar ...,Bot,2018-02-02 13:52:17,1454345038013757@messenger.gw.msging.net,7
...,...,...,...,...,...,...
33760,650953,Diz aí... de qual bairro você quer ver os bloc...,Bot,2018-02-27 09:16:20,1897100100309538@messenger.gw.msging.net,12
33761,650957,Não tenho dúvidas que deu para curtir muito n...,Bot,2018-02-27 09:16:35,1897100100309538@messenger.gw.msging.net,13
33762,650960,Olha os blocos que passaram por lá 👀,Bot,2018-02-27 09:16:36,1897100100309538@messenger.gw.msging.net,14
33764,651749,Chama sua galera para o Cai na Folia.,Bot,2018-02-27 10:17:11,1897100100309538@messenger.gw.msging.net,16


In [75]:
user_messages = message_enxuta[message_enxuta.MessageOriginator == 'User']
user_messages

,MessageSequentialID,Message,MessageOriginator,date,user,message_sequence_number
0,73719,comecar,User,2018-02-02 13:51:58,1454345038013757@messenger.gw.msging.net,1
3,73726,beber,User,2018-02-02 13:52:10,1454345038013757@messenger.gw.msging.net,4
7,73735,MenuAgenda,User,2018-02-02 13:52:24,1454345038013757@messenger.gw.msging.net,8
8,73738,10/02/2018,User,2018-02-02 13:52:35,1454345038013757@messenger.gw.msging.net,9
11,73802,qual o seu nome,User,2018-02-02 13:53:00,1454345038013757@messenger.gw.msging.net,12
...,...,...,...,...,...,...
33749,650925,Começar,User,2018-02-27 09:15:22,1897100100309538@messenger.gw.msging.net,1
33752,650932,tudo,User,2018-02-27 09:15:38,1897100100309538@messenger.gw.msging.net,4
33756,650943,MenuAgenda,User,2018-02-27 09:16:08,1897100100309538@messenger.gw.msging.net,8
33758,650948,08/02/2018,User,2018-02-27 09:16:18,1897100100309538@messenger.gw.msging.net,10


In [76]:
#Tempo de resposta do bot
df_message_detail = message_enxuta.copy()
df_message_detail= df_message_detail.sort_values(by=['user', 'message_sequence_number'], ascending=True).reset_index(drop=True)
df_message_detail['previous_originator'] = df_message_detail.groupby('user')['MessageOriginator'].shift(1)
df_message_detail['previous_datetime'] = df_message_detail.groupby('user')['date'].shift(1)

# Tempo de resposta (delta)
df_message_detail['response_time'] = (df_message_detail['date'] - df_message_detail['previous_datetime']).dt.total_seconds()
df_message_detail = df_message_detail.fillna({'response_time': 0})
df_message_detail

,MessageSequentialID,Message,MessageOriginator,date,user,message_sequence_number,previous_originator,previous_datetime,response_time
0,230227,Começar,User,2018-02-08 18:31:33,1065689913556034@messenger.gw.msging.net,1,NaN,NaT,0.0
1,230229,"E ai Reinaldo Oliveira, o carnaval de BH já co...",Bot,2018-02-08 18:31:35,1065689913556034@messenger.gw.msging.net,2,User,2018-02-08 18:31:33,2.0
2,230231,"Se você não sabe onde ir, posso te ajudar com ...",Bot,2018-02-08 18:31:38,1065689913556034@messenger.gw.msging.net,3,Bot,2018-02-08 18:31:35,3.0
3,230238,beber,User,2018-02-08 18:31:59,1065689913556034@messenger.gw.msging.net,4,Bot,2018-02-08 18:31:38,21.0
4,230240,🎵 Ninguém é de ferro 🍺\n,Bot,2018-02-08 18:32:01,1065689913556034@messenger.gw.msging.net,5,User,2018-02-08 18:31:59,2.0
...,...,...,...,...,...,...,...,...,...
33761,308187,Vem pra folia boa!!! 🎉 \nTe espero lá..,Bot,2018-02-11 09:51:57,999517963506386@messenger.gw.msging.net,11,Bot,2018-02-11 09:51:08,49.0
33762,308189,Quando estiver quase na hora te chamo por aqui...,Bot,2018-02-11 09:51:59,999517963506386@messenger.gw.msging.net,12,Bot,2018-02-11 09:51:57,2.0
33763,308197,A concentração do Bloco dos Dados vai ser na 👉...,Bot,2018-02-11 09:52:22,999517963506386@messenger.gw.msging.net,13,Bot,2018-02-11 09:51:59,23.0
33764,313444,Curtindo o AbacaBloco?🍍🍍\nJunta a galera e man...,Bot,2018-02-11 13:11:13,cainafolia_block_188@broadcast.msging.net,1,NaN,NaT,0.0


In [77]:
# Resposta do USER após o BOT
user_responses = df_message_detail[
    (df_message_detail['MessageOriginator'] == 'User') &
    (df_message_detail['previous_originator'] == 'Bot')
]
user_responses

,MessageSequentialID,Message,MessageOriginator,date,user,message_sequence_number,previous_originator,previous_datetime,response_time
3,230238,beber,User,2018-02-08 18:31:59,1065689913556034@messenger.gw.msging.net,4,Bot,2018-02-08 18:31:38,21.0
7,230247,MenuHoje,User,2018-02-08 18:32:29,1065689913556034@messenger.gw.msging.net,8,Bot,2018-02-08 18:32:06,23.0
9,230255,📅 Agenda,User,2018-02-08 18:33:43,1065689913556034@messenger.gw.msging.net,10,Bot,2018-02-08 18:32:32,71.0
18,377503,Agenda,User,2018-02-13 12:17:34,1065689913556034@messenger.gw.msging.net,19,Bot,2018-02-10 13:34:12,254602.0
23,377525,Mais próximo,User,2018-02-13 12:19:07,1065689913556034@messenger.gw.msging.net,24,Bot,2018-02-13 12:18:58,9.0
...,...,...,...,...,...,...,...,...,...
33740,344447,ads-bot-carnaval-anuncio2,User,2018-02-12 12:38:27,930345150423784@messenger.gw.msging.net,4,Bot,2018-02-12 12:38:25,2.0
33742,344465,MenuEnviarLocal,User,2018-02-12 12:38:51,930345150423784@messenger.gw.msging.net,6,Bot,2018-02-12 12:38:32,19.0
33746,344589,MenuHoje,User,2018-02-12 12:41:28,930345150423784@messenger.gw.msging.net,10,Bot,2018-02-12 12:41:18,10.0
33750,344687,Menu,User,2018-02-12 12:45:40,930345150423784@messenger.gw.msging.net,14,Bot,2018-02-12 12:45:22,18.0


In [78]:
# Análise do tempo de resposta do usuário
print (df_message_detail['response_time'].describe().round(2))
print (df_message_detail['response_time'].quantile([0.25, 0.5, 0.75, 0.80,0.85 ,0.9, 0.95, 0.99]).round(2)) 

#Logo, como segurança vamos estabelecer o treshold de 900 segundos (15 minutos) para considerar o início de uma nova sessão de conversa com o chatbot
treshold_new_session = 900



count      33766.00
mean        7888.41
std        44331.56
min            0.00
25%            1.00
50%            4.00
75%           18.00
max      1482221.00
Name: response_time, dtype: float64
0.25         1.0
0.50         4.0
0.75        18.0
0.80        25.0
0.85        40.0
0.90       145.0
0.95     37728.0
0.99    197051.3
Name: response_time, dtype: float64


### Identificando sessões e callbacks

In [79]:
# Criando uma coluna para identificar sessão por user

def calculate_session(group):
    # A condição para considerar uma nova sessão:
    # Se a nova mensagem  for do usuário E o response_time >= treshold -> caso em que o usuário voltou ao bot organicamente ou voltou um tempo depois do callback do bot
    # OU se a nova mensagem for do usuário E a resposta anterior - callback do bot - foi >= treshold -> caso em que o usuário voltou ao chat logo após o callback do bot
    is_session_break = (
                      ((group['MessageOriginator'].str.lower() == 'user') &
                      (group['response_time']>= treshold_new_session ))
                      | 
                      ((group['MessageOriginator'].str.lower() == 'user') 
                         & (group['response_time'].shift(1) >= treshold_new_session)
                         & (group['MessageOriginator'].shift(1).str.lower() == 'bot'))
    )
    
    # A primeira mensagem de cada usuário SEMPRE inicia uma sessão
    return is_session_break.cumsum() + 1

df_message_detail['user_session_id'] = df_message_detail.groupby('user').apply(calculate_session, include_groups=False).reset_index(drop=True)
df_message_detail['user_ever_interacted'] = df_message_detail.groupby('user')['MessageOriginator'].transform(lambda x: (x == 'User').max())
df_message_detail['user_session_id'] = np.where(df_message_detail['user_ever_interacted']== False, np.nan, df_message_detail['user_session_id'])
df_message_detail['user_session_id'] = df_message_detail['user_session_id'].astype('Int64')  # permite nulos
df_message_detail

,MessageSequentialID,Message,MessageOriginator,date,user,message_sequence_number,previous_originator,previous_datetime,response_time,user_session_id,user_ever_interacted
0,230227,Começar,User,2018-02-08 18:31:33,1065689913556034@messenger.gw.msging.net,1,NaN,NaT,0.0,1,True
1,230229,"E ai Reinaldo Oliveira, o carnaval de BH já co...",Bot,2018-02-08 18:31:35,1065689913556034@messenger.gw.msging.net,2,User,2018-02-08 18:31:33,2.0,1,True
2,230231,"Se você não sabe onde ir, posso te ajudar com ...",Bot,2018-02-08 18:31:38,1065689913556034@messenger.gw.msging.net,3,Bot,2018-02-08 18:31:35,3.0,1,True
3,230238,beber,User,2018-02-08 18:31:59,1065689913556034@messenger.gw.msging.net,4,Bot,2018-02-08 18:31:38,21.0,1,True
4,230240,🎵 Ninguém é de ferro 🍺\n,Bot,2018-02-08 18:32:01,1065689913556034@messenger.gw.msging.net,5,User,2018-02-08 18:31:59,2.0,1,True
...,...,...,...,...,...,...,...,...,...,...,...
33761,308187,Vem pra folia boa!!! 🎉 \nTe espero lá..,Bot,2018-02-11 09:51:57,999517963506386@messenger.gw.msging.net,11,Bot,2018-02-11 09:51:08,49.0,1,True
33762,308189,Quando estiver quase na hora te chamo por aqui...,Bot,2018-02-11 09:51:59,999517963506386@messenger.gw.msging.net,12,Bot,2018-02-11 09:51:57,2.0,1,True
33763,308197,A concentração do Bloco dos Dados vai ser na 👉...,Bot,2018-02-11 09:52:22,999517963506386@messenger.gw.msging.net,13,Bot,2018-02-11 09:51:59,23.0,1,True
33764,313444,Curtindo o AbacaBloco?🍍🍍\nJunta a galera e man...,Bot,2018-02-11 13:11:13,cainafolia_block_188@broadcast.msging.net,1,NaN,NaT,0.0,<NA>,False


In [80]:
# Para análises futuras, vamos sinalizar as mensagens de callback do bot
#Callback do bot é uma mensagem do bot que ocorre após um tempo de resposta >= treshold_new_session, ou seja, o bot envia uma mensagem após um período de inatividade do usuário
#Identificamos inatividade do user a partir de 15 minutos (900 seg) sem interação com o bot
def identify_callback(group):
    # Para se considerar um callback do bot, a mensagem deve ser do bot com um response_time >= treshold_new_session # Indicando que o BOT enviou uma mensagem após um período de inatividade do usuário
    # OU a mensagem é do bot, e a resposta anterior é do bot e com a mensagem anterior com um response_time >= treshold_new_session # ou seja, um callback do bot após o outro, tentando novamente trazer o usuário de volta
    is_bot_callback = (
                      ((group['MessageOriginator'].str.lower() == 'bot') &
                      (group['response_time']>= treshold_new_session ))
                      | 
                      ((group['MessageOriginator'].str.lower() == 'bot')
                       & (group['previous_originator'].str.lower() == 'bot') 
                         & (group['response_time'].shift(1)>= treshold_new_session ))  # Verifica se a mensagem anterior foi do usuário
    )
    
    # A primeira mensagem de cada usuário SEMPRE inicia uma sessão
    return is_bot_callback

df_message_detail['bot_callback'] = df_message_detail.groupby('user').apply(identify_callback, include_groups=False).reset_index(drop=True)
df_message_detail['bot_callback'] = df_message_detail['bot_callback'].astype(int)

In [81]:
# Função para numerar as sessões diárias sequencialmente por usuário
df_message_detail['day'] =df_message_detail['date'].dt.date

def assign_daily_session_ids(group):
    # Seleciona apenas os dias em que o usuário enviou mensagens
    active_days = group[group['MessageOriginator'] == 'User']['day'].drop_duplicates().sort_values()
    
    # Cria um dicionário {data: número_da_sessao}
    daily_session_map = {date: idx + 1 for idx, date in enumerate(active_days)}
    
    # Mapeia de volta para o grupo completo (bot + user)
    group['daily_session_id'] = group['day'].map(daily_session_map)
    return group

# Aplicar por usuário
df_message_detail = df_message_detail.groupby('user').apply(assign_daily_session_ids, include_groups=False).reset_index()
df_message_detail.drop(columns = 'level_1', inplace=True)  # Remove a coluna de nível extra criada pelo groupby
df_message_detail['daily_session_id'] = df_message_detail['daily_session_id'].astype('Int64') 


In [ ]:
#Calculando a sequência de mensagens dentro de cada sessão diária
df_message_detail['session_sequence'] = df_message_detail[df_message_detail.MessageOriginator== 'User'].groupby(['user', 'user_session_id'])['MessageSequentialID'].rank(method='dense').astype('Int64')
df_message_detail['daily_session_sequence'] = df_message_detail[df_message_detail.MessageOriginator== 'User'].groupby(['user', 'daily_session_id'])['MessageSequentialID'].rank(method='dense').astype('Int64')
#Tratamento para preencher valores nulos de sessão, indicando sessão = 0 quando não houve interação do usuário com o bot 
df_message_detail[['daily_session_id','user_session_id' ]] = df_message_detail[['daily_session_id','user_session_id' ]].fillna(0)
df_message_detail

,user,MessageSequentialID,Message,MessageOriginator,date,message_sequence_number,previous_originator,previous_datetime,response_time,user_session_id,user_ever_interacted,bot_callback,day,daily_session_id,session_sequence,daily_session_sequence
0,1065689913556034@messenger.gw.msging.net,230227,Começar,User,2018-02-08 18:31:33,1,NaN,NaT,0.0,1,True,0,2018-02-08,1,1,1
1,1065689913556034@messenger.gw.msging.net,230229,"E ai Reinaldo Oliveira, o carnaval de BH já co...",Bot,2018-02-08 18:31:35,2,User,2018-02-08 18:31:33,2.0,1,True,0,2018-02-08,1,<NA>,<NA>
2,1065689913556034@messenger.gw.msging.net,230231,"Se você não sabe onde ir, posso te ajudar com ...",Bot,2018-02-08 18:31:38,3,Bot,2018-02-08 18:31:35,3.0,1,True,0,2018-02-08,1,<NA>,<NA>
3,1065689913556034@messenger.gw.msging.net,230238,beber,User,2018-02-08 18:31:59,4,Bot,2018-02-08 18:31:38,21.0,1,True,0,2018-02-08,1,2,2
4,1065689913556034@messenger.gw.msging.net,230240,🎵 Ninguém é de ferro 🍺\n,Bot,2018-02-08 18:32:01,5,User,2018-02-08 18:31:59,2.0,1,True,0,2018-02-08,1,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33761,999517963506386@messenger.gw.msging.net,308187,Vem pra folia boa!!! 🎉 \nTe espero lá..,Bot,2018-02-11 09:51:57,11,Bot,2018-02-11 09:51:08,49.0,1,True,0,2018-02-11,1,<NA>,<NA>
33762,999517963506386@messenger.gw.msging.net,308189,Quando estiver quase na hora te chamo por aqui...,Bot,2018-02-11 09:51:59,12,Bot,2018-02-11 09:51:57,2.0,1,True,0,2018-02-11,1,<NA>,<NA>
33763,999517963506386@messenger.gw.msging.net,308197,A concentração do Bloco dos Dados vai ser na 👉...,Bot,2018-02-11 09:52:22,13,Bot,2018-02-11 09:51:59,23.0,1,True,0,2018-02-11,1,<NA>,<NA>
33764,cainafolia_block_188@broadcast.msging.net,313444,Curtindo o AbacaBloco?🍍🍍\nJunta a galera e man...,Bot,2018-02-11 13:11:13,1,NaN,NaT,0.0,0,False,0,2018-02-11,0,<NA>,<NA>


## Identificando comportamentos, tendências e padrões de uso

### Análise de tempo de resposta 

In [83]:
summary_response_time_session = df_message_detail.copy()
#Tratamento para remover mensagens que tem o response_time maior que o treshold_new_session, pois são consideradas novas sessões de conversa e portanto o tempo de resposta não é válido para a análise 
summary_response_time_session ['cleaned_response_time'] = np.where(summary_response_time_session['response_time'] > treshold_new_session, np.nan, summary_response_time_session['response_time'])
summary_response_time_session = summary_response_time_session.dropna(subset=['cleaned_response_time'])
summary_response_time_session = summary_response_time_session.groupby(['user_session_id', 'MessageOriginator']).agg({'MessageSequentialID': 'nunique','response_time':'mean', 'bot_callback':'sum'}).reset_index()
summary_response_time_session.columns = ['user_session_id', 'MessageOriginator', 'message_count','mean_response_time', 'bot_callback_count']
summary_response_time_session['mean_response_time'] = summary_response_time_session['mean_response_time'].round(2)
summary_response_time_session

,user_session_id,MessageOriginator,message_count,mean_response_time,bot_callback_count
0,0,Bot,10,1.30,0
1,1,Bot,11107,8.60,148
2,1,User,5831,23.64,0
3,2,Bot,3124,10.73,67
4,2,User,1336,33.82,0
...,...,...,...,...,...
211,116,Bot,2,1.00,0
212,117,Bot,3,1.33,0
213,117,User,2,39.00,0
214,118,Bot,4,41.00,1


In [107]:
summary_response_time = df_message_detail.copy()
#Tratamento para remover mensagens que tem o response_time maior que o treshold_new_session, pois são consideradas novas sessões de conversa e portanto o tempo de resposta não é válido para a análise 
print("Descritivos do tempo de resposta médio durante uma sessão de conversa")
print (summary_response_time[(summary_response_time['response_time'] < treshold_new_session) & (summary_response_time.MessageOriginator == 'User')& (summary_response_time.response_time > 0)]['response_time'].describe().round(2))
print("Descritivos do tempo de resposta médio entre sessões de conversa - ou seja quanto tempo em média o usuário leva para voltar ao bot")
print (summary_response_time[(summary_response_time['response_time'] >= treshold_new_session) & (summary_response_time.MessageOriginator == 'User')]['response_time'].describe().round(2))
# Exportando os resultados
in_session_analysis = summary_response_time[(summary_response_time['response_time'] < treshold_new_session) & (summary_response_time.MessageOriginator == 'User')& (summary_response_time.response_time > 0)]['response_time'].describe().T.round(2)
between_session_analysis = summary_response_time[(summary_response_time['response_time'] >= treshold_new_session) & (summary_response_time.MessageOriginator == 'User')]['response_time'].describe().T.round(2)
in_session_analysis.to_excel(r'../03. Exported results\in_session_analysis.xlsx')
between_session_analysis.to_excel(r'../03. Exported results\between_session_analysis.xlsx')

Descritivos do tempo de resposta médio durante uma sessão de conversa
count    8507.00
mean       36.63
std        89.01
min         1.00
25%         9.00
50%        14.00
75%        26.00
max       882.00
Name: response_time, dtype: float64
Descritivos do tempo de resposta médio entre sessões de conversa - ou seja quanto tempo em média o usuário leva para voltar ao bot
count       1334.00
mean       72656.47
std       119316.21
min          900.00
25%         6353.00
50%        33162.50
75%        81227.75
max      1482221.00
Name: response_time, dtype: float64


### Análise de Sessões

In [85]:
daily_session_summary = df_message_detail.copy()
daily_session_summary = daily_session_summary.groupby(['daily_session_id']).agg({
    'MessageSequentialID': 'nunique',
    'bot_callback': 'sum',
    'MessageOriginator' : lambda x: (x == 'Bot').sum(),
    'user': 'nunique',
}).reset_index()

daily_session_summary['user_message_count'] = daily_session_summary['MessageSequentialID'] - daily_session_summary['MessageOriginator']
daily_session_summary = daily_session_summary.rename(columns={
    'MessageSequentialID': 'total_message_count',
    'bot_callback': 'bot_callback_count',
    'MessageOriginator': 'bot_message_count'
})
daily_session_summary['messages_per_user'] = (daily_session_summary['total_message_count'] / daily_session_summary['user']).round(0)
daily_session_summary['user_messages_per_user'] = (daily_session_summary['user_message_count'] / daily_session_summary['user']).round(0)
daily_session_summary

,daily_session_id,total_message_count,bot_callback_count,bot_message_count,user,user_message_count,messages_per_user,user_messages_per_user
0,0,758,663,758,449,0,2.0,0.0
1,1,18222,185,11842,1103,6380,17.0,6.0
2,2,6327,445,4324,537,2003,12.0,4.0
3,3,3004,206,2066,247,938,12.0,4.0
4,4,1893,94,1304,132,589,14.0,4.0
5,5,1064,75,743,81,321,13.0,4.0
6,6,710,43,508,52,202,14.0,4.0
7,7,430,36,303,30,127,14.0,4.0
8,8,226,19,157,17,69,13.0,4.0
9,9,440,15,292,12,148,37.0,12.0


In [ ]:
daily_session_summary

In [86]:
session_summary = df_message_detail.copy()
session_summary = session_summary.groupby(['user_session_id']).agg({
    'MessageSequentialID': 'nunique',
    'bot_callback': 'sum',
    'MessageOriginator' : lambda x: (x == 'Bot').sum(),
    'user': 'nunique'
}).reset_index()

session_summary['user_message_count'] = session_summary['MessageSequentialID'] - session_summary['MessageOriginator']
session_summary = session_summary.rename(columns={
    'MessageSequentialID': 'total_message_count',
    'bot_callback': 'bot_callback_count',
    'MessageOriginator': 'bot_message_count'
})
session_summary['messages_per_user'] = (session_summary['total_message_count'] / session_summary['user']).round(0)
session_summary['user_messages_per_user'] = (session_summary['user_message_count'] / session_summary['user']).round(0)
session_summary

,user_session_id,total_message_count,bot_callback_count,bot_message_count,user,user_message_count,messages_per_user,user_messages_per_user
0,0,12,2,12,6,0,2.0,0.0
1,1,17622,832,11791,1103,5831,16.0,5.0
2,2,5108,330,3387,557,1721,9.0,3.0
3,3,2693,185,1871,295,822,9.0,3.0
4,4,1849,108,1265,196,584,9.0,3.0
...,...,...,...,...,...,...,...,...
115,115,3,0,2,1,1,3.0,1.0
116,116,3,0,2,1,1,3.0,1.0
117,117,8,2,5,1,3,8.0,3.0
118,118,6,2,5,1,1,6.0,1.0


In [109]:
#Calculando média de mensages do usuário por sessão e por sessão diária 
daily_user_msg_mean = daily_session_summary['user_messages_per_user'].mean().round(0)
session_user_msg_mean = session_summary['user_messages_per_user'].mean().round(0)
#Calculando média de mensages trocadas entre user e bot por sessão e por sessão diária 
daily_msg_mean = daily_session_summary['messages_per_user'].mean().round(0)
session_msg_mean = session_summary['messages_per_user'].mean().round(0)
#Printando os resultados
print ('Média de mensagens do usuário ao bot por dia: '+ str(int(daily_user_msg_mean)))
print ('Média de mensagens do usuário ao bot por sessão: '+ str(int(session_user_msg_mean)))
print ('Média de mensagens trocadas entre usuário e bot por dia: '+ str(int(daily_msg_mean)))
print ('Média de mensagens trocadas entre usuário e bot por sessão: '+ str(int(session_msg_mean)))

pd.DataFrame({
    'daily_user_msg_mean': [daily_user_msg_mean],
    'session_user_msg_mean': [session_user_msg_mean],
    'daily_msg_mean': [daily_msg_mean],
    'session_msg_mean': [session_msg_mean]
}).to_excel(r'../03. Exported results\message_summary.xlsx', index=False)


Média de mensagens do usuário ao bot por dia: 8
Média de mensagens do usuário ao bot por sessão: 4
Média de mensagens trocadas entre usuário e bot por dia: 24
Média de mensagens trocadas entre usuário e bot por sessão: 12


In [88]:
print ('Total de callbacks do bot: ' + str (df_message_detail[df_message_detail.bot_callback == 1]['MessageSequentialID'].nunique()))
print ('Callbacks do bot sem interação do usuário no dia do callback: ' + str (df_message_detail[(df_message_detail.daily_session_id == 0) & (df_message_detail.bot_callback==1)]['MessageSequentialID'].nunique()))
print ('Callbacks do bot com interação do usuário no dia do callback: ' + str (df_message_detail[(df_message_detail.daily_session_id!= 0) & (df_message_detail.bot_callback==1)]['MessageSequentialID'].nunique()))
print ('Taxa de eficácia do callback do bot: ' + str(round((df_message_detail[(df_message_detail.daily_session_id!= 0) & (df_message_detail.bot_callback==1)]['MessageSequentialID'].nunique())/(df_message_detail[df_message_detail.bot_callback == 1]['MessageSequentialID'].nunique() ) * 100, 2)) + '%')

Total de callbacks do bot: 1804
Callbacks do bot sem interação do usuário no dia do callback: 663
Callbacks do bot com interação do usuário no dia do callback: 1141
Taxa de eficácia do callback do bot: 63.25%


In [89]:
daily_session_summary.to_excel(r'../03. Exported results\daily_session_summary.xlsx', index=False)
session_summary.to_excel(r'../03. Exported results\session_summary.xlsx', index=False)

### Mapeando tendências temporais

In [90]:
#Mapeando as datas do carnaval de 2018
carnaval_dates_map = {
    pd.to_datetime('2018-02-09'): 'Dia 01 do Carnaval',
    pd.to_datetime('2018-02-10'): 'Dia 02 do Carnaval',
    pd.to_datetime('2018-02-11'): 'Dia 03 do Carnaval',
    pd.to_datetime('2018-02-12'): 'Dia 04 do Carnaval',
    pd.to_datetime('2018-02-13'): 'Dia 05 do Carnaval',
    pd.to_datetime('2018-02-14'): 'Ultimo dia do Carnaval',
}

df_message_detail['carnaval_date'] = df_message_detail['date'].dt.normalize().map(carnaval_dates_map)
df_message_detail['carnaval_date'] = np.where(df_message_detail['date'] < '2018-02-09', 'Pré Carnaval',
                                                np.where(df_message_detail['date'] > '2018-02-14', 'Pós Carnaval',
                                                         df_message_detail['carnaval_date']))

In [91]:
#Mapeando hora do dia 
hour_bins = [0, 6, 12, 18, 24]
hour_labels = ['Madrugada', 'Manhã', 'Tarde', 'Noite']
# Criar a coluna 'hour_of_day' com base na hora da data
df_message_detail['hour_of_day'] = pd.cut(df_message_detail['date'].dt.hour, bins=hour_bins, labels=hour_labels, right=False,include_lowest=True)
df_message_detail

,user,MessageSequentialID,Message,MessageOriginator,date,message_sequence_number,previous_originator,previous_datetime,response_time,user_session_id,user_ever_interacted,bot_callback,day,daily_session_id,session_sequence,daily_session_sequence,carnaval_date,hour_of_day
0,1065689913556034@messenger.gw.msging.net,230227,Começar,User,2018-02-08 18:31:33,1,NaN,NaT,0.0,1,True,0,2018-02-08,1,1,1,Pré Carnaval,Noite
1,1065689913556034@messenger.gw.msging.net,230229,"E ai Reinaldo Oliveira, o carnaval de BH já co...",Bot,2018-02-08 18:31:35,2,User,2018-02-08 18:31:33,2.0,1,True,0,2018-02-08,1,<NA>,<NA>,Pré Carnaval,Noite
2,1065689913556034@messenger.gw.msging.net,230231,"Se você não sabe onde ir, posso te ajudar com ...",Bot,2018-02-08 18:31:38,3,Bot,2018-02-08 18:31:35,3.0,1,True,0,2018-02-08,1,<NA>,<NA>,Pré Carnaval,Noite
3,1065689913556034@messenger.gw.msging.net,230238,beber,User,2018-02-08 18:31:59,4,Bot,2018-02-08 18:31:38,21.0,1,True,0,2018-02-08,1,2,2,Pré Carnaval,Noite
4,1065689913556034@messenger.gw.msging.net,230240,🎵 Ninguém é de ferro 🍺\n,Bot,2018-02-08 18:32:01,5,User,2018-02-08 18:31:59,2.0,1,True,0,2018-02-08,1,<NA>,<NA>,Pré Carnaval,Noite
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33761,999517963506386@messenger.gw.msging.net,308187,Vem pra folia boa!!! 🎉 \nTe espero lá..,Bot,2018-02-11 09:51:57,11,Bot,2018-02-11 09:51:08,49.0,1,True,0,2018-02-11,1,<NA>,<NA>,Dia 03 do Carnaval,Manhã
33762,999517963506386@messenger.gw.msging.net,308189,Quando estiver quase na hora te chamo por aqui...,Bot,2018-02-11 09:51:59,12,Bot,2018-02-11 09:51:57,2.0,1,True,0,2018-02-11,1,<NA>,<NA>,Dia 03 do Carnaval,Manhã
33763,999517963506386@messenger.gw.msging.net,308197,A concentração do Bloco dos Dados vai ser na 👉...,Bot,2018-02-11 09:52:22,13,Bot,2018-02-11 09:51:59,23.0,1,True,0,2018-02-11,1,<NA>,<NA>,Dia 03 do Carnaval,Manhã
33764,cainafolia_block_188@broadcast.msging.net,313444,Curtindo o AbacaBloco?🍍🍍\nJunta a galera e man...,Bot,2018-02-11 13:11:13,1,NaN,NaT,0.0,0,False,0,2018-02-11,0,<NA>,<NA>,Dia 03 do Carnaval,Tarde


In [92]:
#Resumo de mensagens por dia do carnaval
carnaval_summary = df_message_detail.copy()
carnaval_summary = carnaval_summary.groupby(['carnaval_date']).agg({
    'MessageSequentialID': 'nunique',
    'bot_callback': 'sum',
    'MessageOriginator' : lambda x: (x == 'Bot').sum(),
    'user': 'nunique'
}).reset_index()

carnaval_summary['user_message_count'] = carnaval_summary['MessageSequentialID'] - carnaval_summary['MessageOriginator']
carnaval_summary = carnaval_summary.rename(columns={
    'MessageSequentialID': 'total_message_count',
    'bot_callback': 'bot_callback_count',
    'MessageOriginator': 'bot_message_count'
})
carnaval_summary

,carnaval_date,total_message_count,bot_callback_count,bot_message_count,user,user_message_count
0,Dia 01 do Carnaval,3138,144,2070,233,1068
1,Dia 02 do Carnaval,4803,444,3331,495,1472
2,Dia 03 do Carnaval,3869,104,2699,231,1170
3,Dia 04 do Carnaval,4783,286,3298,389,1485
4,Dia 05 do Carnaval,3967,146,2645,310,1322
5,Pré Carnaval,11155,311,7301,440,3854
6,Pós Carnaval,2051,369,1400,359,651


In [93]:
#Resumo de mensagens por hora do dia
hour_of_day_summary = df_message_detail.copy()
hour_of_day_summary = hour_of_day_summary.groupby(['hour_of_day']).agg({
    'MessageSequentialID': 'nunique',
    'bot_callback': 'sum',
    'MessageOriginator' : lambda x: (x == 'Bot').sum(),
    'user': 'nunique'
}).reset_index()

hour_of_day_summary['user_message_count'] = hour_of_day_summary['MessageSequentialID'] - hour_of_day_summary['MessageOriginator']
hour_of_day_summary = hour_of_day_summary.rename(columns={
    'MessageSequentialID': 'total_message_count',
    'bot_callback': 'bot_callback_count',
    'MessageOriginator': 'bot_message_count'
})
hour_of_day_summary

C:\Users\diniz\AppData\Local\Temp\ipykernel_8072\4092837785.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  hour_of_day_summary = hour_of_day_summary.groupby(['hour_of_day']).agg({


,hour_of_day,total_message_count,bot_callback_count,bot_message_count,user,user_message_count
0,Madrugada,2176,31,1503,106,673
1,Manhã,8429,467,5756,545,2673
2,Tarde,13556,1126,9144,858,4412
3,Noite,9605,180,6341,520,3264


In [94]:
carnaval_summary.to_excel(r'../03. Exported results\carnaval_summary.xlsx', index=False)
hour_of_day_summary.to_excel(r'../03. Exported results\hour_of_day_summary.xlsx', index=False)

### Identificando padrões no fluxo de mensagens

In [95]:
# Distribuição da quantidade de mensagens por chat com usuário -> calculando a distribuição da quantidade de interações totais de cada usuário com o bot 
#Inclui mensagens do user e do bot durante todas as sessões de conversa
max_user_msg_seq = df_message_detail.groupby('user')['message_sequence_number'].max().reset_index()
print (max_user_msg_seq['message_sequence_number'].describe().round(2))
print (max_user_msg_seq['message_sequence_number'].quantile(0.9).round(2))
bins_msgs = [0, 5, 10,20 ,30, 50, 100, np.inf]
labels_msgs = ['0-5', '6-10', '11-20','21-30' ,'31-50', '51-100', '101+']
message_dist = pd.cut(max_user_msg_seq['message_sequence_number'], bins=bins_msgs,labels =labels_msgs , right=False, include_lowest=True).value_counts().reset_index().sort_values(by = 'message_sequence_number', ascending=True).reset_index(drop=True)
message_dist['percentage'] = (message_dist['count'] / message_dist['count'].sum() * 100).round(2)
message_dist

count    1109.00
mean       30.45
std        54.15
min         1.00
25%        11.00
50%        19.00
75%        35.00
max      1465.00
Name: message_sequence_number, dtype: float64
62.0


,message_sequence_number,count,percentage
0,0-5,60,5.41
1,6-10,162,14.61
2,11-20,353,31.83
3,21-30,197,17.76
4,31-50,172,15.51
5,51-100,121,10.91
6,101+,44,3.97


In [96]:
#Conferindo se a distribuição de mensagens está correta com base no total de users unicos na tabela original de mensagens
print (message_dist['count'].sum())
print (message_trat['user'].nunique())


1109
1109


In [97]:
df_message_detail

,user,MessageSequentialID,Message,MessageOriginator,date,message_sequence_number,previous_originator,previous_datetime,response_time,user_session_id,user_ever_interacted,bot_callback,day,daily_session_id,session_sequence,daily_session_sequence,carnaval_date,hour_of_day
0,1065689913556034@messenger.gw.msging.net,230227,Começar,User,2018-02-08 18:31:33,1,NaN,NaT,0.0,1,True,0,2018-02-08,1,1,1,Pré Carnaval,Noite
1,1065689913556034@messenger.gw.msging.net,230229,"E ai Reinaldo Oliveira, o carnaval de BH já co...",Bot,2018-02-08 18:31:35,2,User,2018-02-08 18:31:33,2.0,1,True,0,2018-02-08,1,<NA>,<NA>,Pré Carnaval,Noite
2,1065689913556034@messenger.gw.msging.net,230231,"Se você não sabe onde ir, posso te ajudar com ...",Bot,2018-02-08 18:31:38,3,Bot,2018-02-08 18:31:35,3.0,1,True,0,2018-02-08,1,<NA>,<NA>,Pré Carnaval,Noite
3,1065689913556034@messenger.gw.msging.net,230238,beber,User,2018-02-08 18:31:59,4,Bot,2018-02-08 18:31:38,21.0,1,True,0,2018-02-08,1,2,2,Pré Carnaval,Noite
4,1065689913556034@messenger.gw.msging.net,230240,🎵 Ninguém é de ferro 🍺\n,Bot,2018-02-08 18:32:01,5,User,2018-02-08 18:31:59,2.0,1,True,0,2018-02-08,1,<NA>,<NA>,Pré Carnaval,Noite
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33761,999517963506386@messenger.gw.msging.net,308187,Vem pra folia boa!!! 🎉 \nTe espero lá..,Bot,2018-02-11 09:51:57,11,Bot,2018-02-11 09:51:08,49.0,1,True,0,2018-02-11,1,<NA>,<NA>,Dia 03 do Carnaval,Manhã
33762,999517963506386@messenger.gw.msging.net,308189,Quando estiver quase na hora te chamo por aqui...,Bot,2018-02-11 09:51:59,12,Bot,2018-02-11 09:51:57,2.0,1,True,0,2018-02-11,1,<NA>,<NA>,Dia 03 do Carnaval,Manhã
33763,999517963506386@messenger.gw.msging.net,308197,A concentração do Bloco dos Dados vai ser na 👉...,Bot,2018-02-11 09:52:22,13,Bot,2018-02-11 09:51:59,23.0,1,True,0,2018-02-11,1,<NA>,<NA>,Dia 03 do Carnaval,Manhã
33764,cainafolia_block_188@broadcast.msging.net,313444,Curtindo o AbacaBloco?🍍🍍\nJunta a galera e man...,Bot,2018-02-11 13:11:13,1,NaN,NaT,0.0,0,False,0,2018-02-11,0,<NA>,<NA>,Dia 03 do Carnaval,Tarde


In [98]:
#Mapeando a sequência de mensagens de usuários mais frequentes a cada sessão
# Dessa forma conseguimos identificar qual caminho mais frequente os usuários tomam ao interagir a cada novo dia com o bot 
message_flow_analysis_daily = df_message_detail[df_message_detail.MessageOriginator=='User'].copy()
message_flow_analysis_daily = message_flow_analysis_daily.groupby(['daily_session_sequence', 'Message']).agg({
    'MessageSequentialID': 'nunique',
    'user':'nunique'}).sort_values(by=['daily_session_sequence', 'MessageSequentialID'], ascending=[True, False]).reset_index()
message_flow_analysis_daily = message_flow_analysis_daily.groupby('daily_session_sequence', ).apply(
    lambda x: x.nlargest(5, 'MessageSequentialID'), include_groups=False).reset_index()
message_flow_analysis_daily.drop(columns='level_1', inplace=True)
message_flow_analysis_daily

,daily_session_sequence,Message,MessageSequentialID,user
0,1,Começar,477,465
1,1,📅 Agenda,268,170
2,1,ads-bot-carnaval,161,151
3,1,ads-bot-carnaval-anuncio3,100,100
4,1,ads-bot-carnaval-anuncio2,92,91
...,...,...,...,...
393,95,❤️,1,1
394,96,👎,1,1
395,97,playlist,1,1
396,98,Obrigado,1,1


In [110]:
#Mapeando a sequência de mensagens de usuários mais frequentes a cada sessão antes do timeout (900 segundos)
# Dessa forma conseguimos identificar qual caminho mais frequente os usuários tomam ao interagir a cada nova sessão de conversa com o bot
message_flow_analysis_session = df_message_detail[df_message_detail.MessageOriginator=='User'].copy()
message_flow_analysis_session = message_flow_analysis_session.groupby(['session_sequence', 'Message']).agg({
    'MessageSequentialID': 'nunique',
    'user':'nunique'}).sort_values(by=['session_sequence', 'MessageSequentialID'], ascending=[True, False]).reset_index()
message_flow_analysis_session = message_flow_analysis_session.groupby('session_sequence', ).apply(
    lambda x: x.nlargest(1, 'MessageSequentialID'), include_groups=False).reset_index()
message_flow_analysis_session.drop(columns='level_1', inplace=True)
message_flow_analysis_session

,session_sequence,Message,MessageSequentialID,user
0,1,Começar,486,467
1,2,dancar,330,328
2,3,MenuHoje,408,391
3,4,MenuHoje,107,100
4,5,📅 Agenda,63,55
...,...,...,...,...
68,69,tchau,1,1
69,70,by,1,1
70,71,exit,1,1
71,72,shutdown,1,1


In [100]:
#Entendendo como os usuários tiveram a primeira interação com o bot
message_flow_analysis = df_message_detail[df_message_detail.message_sequence_number==1].copy()
message_flow_analysis = message_flow_analysis.groupby(['Message']).agg({
    'user':'nunique'}).sort_values(by=['user'], ascending=[False]).reset_index()
message_flow_analysis.rename(columns={'user': 'user_count'}, inplace=True)
message_flow_analysis['importance'] = (message_flow_analysis['user_count'] / message_flow_analysis['user_count'].sum() * 100).round(2)
message_flow_analysis['acum_sum'] = message_flow_analysis['importance'].cumsum().round(2)
message_flow_analysis

,Message,user_count,importance,acum_sum
0,Começar,461,41.57,41.57
1,ads-bot-carnaval,141,12.71,54.28
2,ads-bot-carnaval-anuncio3,95,8.57,62.85
3,ads-bot-carnaval-anuncio2,91,8.21,71.06
4,📅 Agenda,58,5.23,76.29
...,...,...,...,...
130,simm,1,0.09,99.61
131,É claro! 🎉,1,0.09,99.70
132,teste000-broadblip quero-cupom-cabify,1,0.09,99.79
133,😊😊😊😊,1,0.09,99.88


In [101]:
message_flow_analysis.to_excel(r'../03. Exported results\message_flow_analysis.xlsx', index=False)
message_flow_analysis_daily.to_excel(r'../03. Exported results\message_flow_analysis_daily.xlsx', index=False)
message_flow_analysis_session.to_excel(r'../03. Exported results\message_flow_analysis_session.xlsx', index=False)
message_dist.to_excel(r'../03. Exported results\message_dist.xlsx', index=False)

### Análise de Satisfação

In [102]:
# Pesquisa de satisfação:
print ('Total de respostas à pesquisa de satisfação: '+ str (event_tracks_short[(event_tracks_short.Category=='broad-Pesquisa_Satisfacao')].Action.count()))
print ('Percentual de respostas à pesquisa de satisfação: ' + str(round((event_tracks_short[(event_tracks_short.Category=='broad-Pesquisa_Satisfacao')].Action.count() / message_enxuta['user'].nunique()) * 100, 2)) + '%')
df_pesquisa_satisf = event_tracks_short[(event_tracks_short.Category=='broad-Pesquisa_Satisfacao')].Action.value_counts().reset_index()
df_pesquisa_satisf['percent'] = (df_pesquisa_satisf['count'] / df_pesquisa_satisf['count'].sum() * 100).round(2)
df_pesquisa_satisf


Total de respostas à pesquisa de satisfação: 286
Percentual de respostas à pesquisa de satisfação: 25.79%


,Action,count,percent
0,broad-5 Amei,179,62.59
1,broad-4 Curti,81,28.32
2,broad-2 nao curti,11,3.85
3,broad-1 nao gostei,8,2.80
4,broad-3 nao indiferente,7,2.45


In [103]:
df_pesquisa_satisf.to_excel(r'../03. Exported results\pesquisa_satisfacao.xlsx', index=False)
df_message_detail.to_excel(r'../02. Processed data\df_message_detail.xlsx', index=False)